In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import  cross_validation, tree
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Загрузка файла

In [2]:
File = r'C:\Users\Administrator\1RTK\Off.csv'

In [3]:
Off = pd.read_csv(File, sep=';', encoding='cp1251')
Off.head()

,USER_ID,PHONE,DATE_BEGIN,DATE_END,COD,NAME
0,143779,NaN,01.03.1997,21.07.2015,20155,ПВ Пользование абонентской радиоточкой
1,143779,NaN,01.10.2014,15.07.2015,1470,ГТС Тональный набор
2,143779,NaN,01.10.2014,15.07.2015,1469,ГТС Уведомление о поступлении нового вызова
3,143780,3842524316,01.10.2013,21.01.2016,203403,ГТС Пред.в пост.польз.абон.линии с инд.схемой ...
4,143780,NaN,01.10.2014,21.01.2016,1469,ГТС Уведомление о поступлении нового вызова


### Выделение услуг в отдельную таблицу

In [4]:
#поиск частых услуг
drop = Off.drop_duplicates(['COD'], keep='last')
name = drop[['COD','NAME']]
name= name.reset_index(level='NA')
del name['index']
name = name.set_index(['COD'])
name

,NAME
COD,
97728,*Добровольная блокировка порта XDSL
40570,ГТС Пред.впольз.аб.лин.с спар.сх.с аб.сист.опл...
40733,"ГТС ""0""внутри сети2016 при пред.мест.тел.соед,..."
97665,Выгодное летоИнтернет по GPON до 25 Мбит/с
10143,*Срочная сделка на 2 года с пролонгацией
40629,Пред.150мин ВЗсоед.на абон.DEF (Пакет5 при СПУ...
40681,"СТС Опц.""0""внутри сети при пред.мест.тел.соед,..."
70446,"Интернет по ETTH Бизнес Эксперт Drive, безлими..."
104320,ПД Услуги по тех.обслуживанию и экспл.оборудов...


In [209]:
#Услуги
data = pd.value_counts(Off['COD'].values)
freq = pd.DataFrame(data, columns = ['FREQ'])
freq

,FREQ
203403,95002
53105,78660
81301,56035
1469,50841
1470,37136
203401,36389
2021,35816
80551,32428
203402,32223
53104,27867


In [210]:
# услуги
survice = name.join(freq)
survice.sort_values(by='FREQ', ascending=False)

,NAME,FREQ
COD,,
203403,ГТС Пред.в пост.польз.абон.линии с инд.схемой ...,95002
53105,*Получение счета/расшифровок Абонентом в подра...,78660
81301,Предоставление в пользование оборудования ONT(...,56035
1469,ГТС Уведомление о поступлении нового вызова,50841
1470,ГТС Тональный набор,37136
203401,ГТС Пред.в пост.польз.абон.линии с инд.схемой ...,36389
2021,ГТС Предост. мест. соед. абонентская система о...,35816
80551,"SDP IPTV Предоставление ТВ-приставки ""Стандарт""",32428
203402,ГТС Пред.в пост.польз.абон.линии с инд.схемой ...,32223


In [211]:
#Услуги
survice = survice.sort_index()

survice.to_csv('Survice.csv',sep=';')

Редактирование данных, исправление ошибок 

In [71]:
Off = Off.drop(['PHONE'], axis =1)

In [207]:
#исправлена ошибка в дате
err = Off['DATE_BEGIN'].str.contains(':')
Err = err==True
err[Err]
Off.loc[1030554,:]
Off.loc[1030554,'DATE_BEGIN']='29.10.2009'

In [ ]:
Off['DATE_END']=Off['DATE_END'].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))
Off['DATE_BEGIN']=Off['DATE_BEGIN'].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))

### Таблица ответов

In [42]:
Off2 = Off.drop(['PHONE', 'DATE_BEGIN', 'DATE_END','NAME'], axis =1)

# Выбираем необходимые услуги по коду
serv = (203403, 81301, 203401, 2021, 80551, 203402, 40520, 81302, 81668)

num = Off2['COD'].size

for i in range(0,num):
    if Off2.loc[i,'COD'] in serv:
        Off2.set_value(i,'OFF', 1)
    else: Off2.set_value(i,'OFF', 0)
        
Off2 = Off2.groupby('USER_ID').max()

Off2 = Off2.drop(['COD'], axis =1)

Off2['OFF'] = Off2.OFF.astype(int)

Off2

,OFF
USER_ID,
143779,0.0
143780,1.0
143789,0.0
143797,0.0
143802,1.0
143804,1.0
143809,1.0
143812,1.0
143815,0.0


In [16]:
Off2.to_csv('Off_Y.csv',sep=';')

### СВЕРКА 2 ТАБЛИЦ

In [20]:
File1 = r'C:\Users\Administrator\1RTK\Off_Y.csv'
File2 = r'C:\Users\Administrator\1RTK\Pay_days124.csv'

In [21]:
y = pd.read_csv(File1, sep=';', encoding='cp1251')
y.head()

,USER_ID,OFF
0,143779,0
1,143780,1
2,143789,0
3,143797,0
4,143802,1


In [22]:
X = pd.read_csv(File2, sep=';', encoding='cp1251')
X.head()

,USER_ID,1,2,3,4,5,6,7,8,9,...,M17,M18,M19,M20,M21,M22,MIN,MAX,SUM,MED
0,143779,0,1,0,0,1,1,3,2,4,...,0,1,0,0,2,0,0.0,1000.0,8963.80,199.0
1,143780,2,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0.0,500.0,4341.00,200.0
2,143789,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,190.0,511.0,8065.00,257.0
3,143797,0,0,0,0,0,0,0,0,1,...,0,0,0,0,2,0,0.0,313.0,7158.38,243.0
4,143802,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0.0,1500.0,5825.70,0.0


In [23]:
#Удаляем строки с разными ID
y = y.loc[y['USER_ID'].isin(X['USER_ID'])]
X = X.loc[X['USER_ID'].isin(y['USER_ID'])] 

In [24]:
y = y.sort_values(by='USER_ID')
X = X.sort_values(by='USER_ID')

In [25]:
y = y.drop(['USER_ID'], axis =1)
X = X.drop(['USER_ID'], axis =1)

In [27]:
# убираем мешающие результату столбцы. Сумма, макс, медиана, дебит. задолженность
X = X.drop(['SUM','MED','MAX', 'M13', 'M14', 'M15', 'M16'], axis=1)

### Логистическая регрессия

In [28]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.3)

In [29]:
y_train = y_train['OFF'].ravel()
scaler = StandardScaler()

x_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),)
x_test_scaled = pd.DataFrame(scaler.fit_transform(X_test),)

In [30]:
cv = KFold(n=len(y_train), n_folds=5, shuffle=True, random_state=42)

param_grid = { 'C': [1e-5, 1e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 1, 10] }

clf = LogisticRegression(penalty='l2', random_state=42)

gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', cv=cv, verbose=2, n_jobs=-1)

gs.fit(x_train_scaled, y_train)

print('Наилучшее качество:', gs.best_score_)

gs.grid_scores_

gs.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.6min finished


Наилучшее качество: 0.62446053748


LogisticRegression(C=1e-05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
#вес коэффициентов
sorted(dict(zip(X_train.columns, gs.best_estimator_.coef_[0])).items(), key=lambda x: -abs(x[1]))

[('M7', 0.072408028209407352),
 ('M21', 0.035685557023690279),
 ('20', 0.030848506970962985),
 ('23', 0.030234791448838512),
 ('M11', -0.028292034000005382),
 ('24', 0.027736178673057128),
 ('19', 0.027482546457260996),
 ('22', 0.026786777558916446),
 ('21', 0.025226036778993731),
 ('18', 0.022932743018099187),
 ('17', 0.022094255349906177),
 ('25', 0.021568672451391178),
 ('16', 0.01943092115973408),
 ('M1', -0.018622650974715788),
 ('15', 0.01824018618815261),
 ('M3', -0.017421695226081797),
 ('13', 0.01729492854963682),
 ('26', 0.017064470897140201),
 ('14', 0.016785397455245189),
 ('27', 0.014904853930597943),
 ('10', 0.013351241759830235),
 ('M6', 0.013124511200499293),
 ('M20', 0.012677174557078437),
 ('28', 0.011429455893985944),
 ('12', 0.010911848809376642),
 ('29', 0.010769901110701112),
 ('11', 0.010234521969175943),
 ('1', 0.0096719735347407),
 ('M2', -0.00889625703127115),
 ('2', 0.0080741223428205094),
 ('3', 0.0078561472621164301),
 ('M17', -0.0078429388460144762),
 ('M1

### Деревья решений

In [22]:
clf_dt = tree.DecisionTreeClassifier(max_depth=10)
clf_dt.fit (X_train_scaled, y_train)
clf_dt.score (X_test_scaled, y_test)

0.70974788049313242

### Gradient Boosting Classifier

In [53]:
gbc = GradientBoostingClassifier(n_estimators=150, max_depth=4, loss='deviance')
gbc.fit(x_train_scaled, y_train)
gbc.score(x_test_scaled, y_test)

0.70553971838633645

### Random Forest Classifier

In [50]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=4, criterion='entropy')
rfc.fit(x_train_scaled, y_train)
rfc.score(x_test_scaled, y_test)                      

0.69961126537088458

TEST

In [33]:
user = Off['USER_ID']< 143816
User = Off[user]
User

,USER_ID,PHONE,DATE_BEGIN,DATE_END,COD,NAME
0,143779,NaN,01.03.1997,21.07.2015,20155,ПВ Пользование абонентской радиоточкой
1,143779,NaN,01.10.2014,15.07.2015,1470,ГТС Тональный набор
2,143779,NaN,01.10.2014,15.07.2015,1469,ГТС Уведомление о поступлении нового вызова
3,143780,3842524316,01.10.2013,21.01.2016,203403,ГТС Пред.в пост.польз.абон.линии с инд.схемой ...
4,143780,NaN,01.10.2014,21.01.2016,1469,ГТС Уведомление о поступлении нового вызова
5,143780,NaN,01.10.2014,21.01.2016,1470,ГТС Тональный набор
6,143780,NaN,30.12.2010,22.01.2016,53105,*Получение счета/расшифровок Абонентом в подра...
7,143789,NaN,01.07.2013,27.01.2016,210010,*Клиент пульта охраны
8,143797,NaN,01.10.2014,12.02.2015,1469,ГТС Уведомление о поступлении нового вызова
9,143802,NaN,01.10.2014,09.03.2016,1470,ГТС Тональный набор


In [34]:
User = User.drop(['PHONE', 'DATE_BEGIN', 'DATE_END','NAME'], axis =1)

In [22]:
User.head()

,USER_ID,COD
0,143779,20155
1,143779,1470
2,143779,1469
3,143780,203403
4,143780,1469


In [35]:
serv = (203403, 81301, 203401, 2021, 80551, 203402, 40520, 81302, 81668)
serv

(203403, 81301, 203401, 2021, 80551, 203402, 40520, 81302, 81668)

In [36]:
num = User['COD'].size

for i in range(0,num):
    if User.loc[i,'COD'] in serv:
        User.set_value(i,'OFF', 1)
    else: User.set_value(i,'OFF', 0)

User

,USER_ID,COD,OFF
0,143779,20155,0.0
1,143779,1470,0.0
2,143779,1469,0.0
3,143780,203403,1.0
4,143780,1469,0.0
5,143780,1470,0.0
6,143780,53105,0.0
7,143789,210010,0.0
8,143797,1469,0.0
9,143802,1470,0.0


In [37]:
User = User.groupby('USER_ID').max()

In [39]:
User = User.drop(['COD'], axis =1)

In [45]:
User['OFF'] = User.OFF.astype(int)


In [46]:
User

,OFF
USER_ID,
143779,0
143780,1
143789,0
143797,0
143802,1
143804,1
143809,1
143812,1
143815,0
